# Weierstrass ℘ Playground — Interactive Trajectory Visualization

This notebook visualizes complex fields derived from the Weierstrass ℘ function on a rectangular lattice and overlays shared second-order trajectories. Choose between:

- **Two-panel mode**: ℘(z) and ℘′(z) with color mapping
- **Three-panel mode**: ℘(z), Re(℘′(z)), and Im(℘′(z)) with grayscale
- **Trajectories**: Follow the second-order ODE z''(t) = -℘(z(t)) z(t)

## Important: How to Use This Notebook

**To avoid errors, please run all cells in order from top to bottom.** 

You can either:
1. Use **Cell → Run All** from the menu, or
2. Run each cell individually using **Shift+Enter**

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import our libraries
from weierstrass_lib import *
from weierstrass_ui import create_ui

# Set matplotlib to inline mode
%matplotlib inline

# Configure matplotlib for high quality plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

## Interactive Controls

The UI and mathematical functions are now modularized in separate Python files:
- `weierstrass_lib.py` - All mathematical computations and visualization functions
- `weierstrass_ui.py` - All UI widgets and layout code

This provides better code organization, reusability, and maintainability.

In [ ]:
# Create and display the UI
ui = create_ui()
ui.display()

## Output Area

Visualization results will appear below:

In [ ]:
# Display the output widget for visualizations
display(ui.get_output_widget())